In [35]:
import os
import random
import sys
import time
import math

import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from scipy.stats import mode

import torch
import torch.nn as nn

from model import ShapeletGenerator, pairwise_dist

np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

def gram_matrix(mat):
  mat = mat.squeeze(dim=0)
  mat = torch.mm(mat, mat.t())
  return mat


def pairwise_dist(x, y):
  x_norm = (x.norm(dim=2)[:, :, None])
  y_t = y.permute(0, 2, 1).contiguous()
  y_norm = (y.norm(dim=2)[:, None])
  y_t = torch.cat([y_t] * x.shape[0], dim=0)
  dist = x_norm + y_norm - 2.0 * torch.bmm(x, y_t)
  return torch.clamp(dist, 0.0, np.inf)

class ShapeletGenerator(nn.Module):

    def __init__(self, n_prototypes, bag_size, n_classes, features):
        n_prototypes = int(n_prototypes)
        super(ShapeletGenerator, self).__init__()

        number_of_rows = features.shape[0]

        random_indices = np.random.choice(number_of_rows, 
                                          size=1, 
                                         replace=False)
        
        prot = features[random_indices, :]
        prot = prot.reshape(1, n_prototypes, prot.shape[1])
        prot = prot.astype("float32")
        self.prototypes = torch.from_numpy(prot).requires_grad_()
        #self.prototypes = (torch.randn(
        #    (1, n_prototypes, bag_size))).requires_grad_()
        if n_classes == 2:
            n_classes = 1
        self.linear_layer = torch.nn.Linear(3 * n_prototypes, n_classes, bias=False)
        #self.linear_layer.weight = torch.nn.Parameter(self.linear_layer.weight/100000)
        self.n_classes = n_classes

    def pairwise_distances(self, x, y):
        #x_norm = (x.norm(dim=2)[:, :, None])
        #y_t = y.permute(0, 2, 1).contiguous()
        #y_norm = (y.norm(dim=2)[:, None])
        #y_t = torch.cat([y_t] * x.shape[0], dim=0)
        #dist = x_norm + y_norm - 2.0 * torch.bmm(x, y_t)
        dist = ((x - y).norm(dim=2)[:, None]).permute(0, 2, 1).contiguous()
        return torch.clamp(dist, 0.0, np.inf)

    def get_output(self, batch_inp):
        dist = self.pairwise_distances(batch_inp, self.prototypes)
        min_dist = dist.min(dim=1)[0]
        max_dist = dist.max(dim=1)[0]
        mean_dist = dist.mean(dim=1)
        all_features = torch.cat([min_dist, max_dist, mean_dist], dim=1)
        logits = self.linear_layer(all_features)

        return logits, all_features

    def forward(self, x):
        logits, distances = self.get_output(x)
        if self.n_classes == 1:
          logits = logits.view(1)
        return logits, distances


def convert_to_bags(data,
                    split_instances=False,
                    instance_norm=False,
                    split_ratio=0.2,
                    stride_ratio=0.5):
  bags = []
  labels = []
  current_bag = []
  current_label = data[0, 0]
  cur = data[0, 1]
  instance_size = np.round(split_ratio * data[0, 2:].shape[0]).astype("int")
  stride = np.round(stride_ratio * instance_size).astype("int")

  for i in range(data.shape[0]):
    if data[i, 1] == cur:
      instance = data[i, 2:]
      if instance_norm:
        instance = (instance - np.mean(instance)) / (1e-08 + np.std(instance))
      if split_instances:
        size = instance.shape[0]
        window = instance_size
        while True:
          current_bag.append(instance[window - instance_size:window])
          window += stride
          if window >= size:
            window = size
            current_bag.append(instance[window - instance_size:window])
            break
      else:
        current_bag.append(instance)
    else:
      bags.append(np.array(current_bag))
      labels.append(np.array(current_label))
      current_label = data[i, 0]
      current_bag = []
      instance = data[i, 2:]
      if instance_norm:
        instance = (instance - np.mean(instance)) / (1e-08 + np.std(instance))
      if split_instances:
        size = instance.shape[0]
        window = instance_size
        while True:
          current_bag.append(instance[window - instance_size:window])
          window += stride
          if window >= size:
            window = size
            current_bag.append(instance[window - instance_size:window])
            break
      else:
        current_bag.append(instance)
      cur = data[i, 1]
  bags.append(np.array(current_bag))
  labels.append(np.array(current_label, dtype="int32"))
  return bags, labels

def find_prototype(bags,
                   features,
                   labels,
                   early_stopping_round = 10):
    n_epochs=100
    batch_size=1
    display_every=5
    final_vals = []
    reg_lambda_dist = generate_random(parameters[0][0], parameters[0][1])
    reg_lambda_w = generate_random(parameters[1][0], parameters[1][1])
    reg_lambda_p = generate_random(parameters[2][0], parameters[2][1])
    lr_prot = generate_random(parameters[3][0], parameters[3][1])
    lr_weights = generate_random(parameters[4][0], parameters[4][1])
    reg_w = 1
    n_prototypes = 1
    #reg_lambda_dist = 0.0005
    #reg_lambda_w = 0.005
    #reg_lambda_p = 0.00005
    #lr_prot = 0.00001
    #lr_weights = 0.00001
    #reg_w = 1
    #n_prototypes = 2
    #n_prototypes = n_prototypes*2
    
    data1 = np.vstack((labels, bags)).T
    data = np.concatenate([data1, features], axis=1)
    n_classes = len(np.unique(labels))
    bags_train, labels_train = convert_to_bags(data)
    bags_train = np.array(bags_train)
    labels_train = np.array(labels_train)

    for rep in range(1, 2):
        vals = []
        for fold in range(1, 2):
            accs = [] 

            use_cuda = False

            bag_size = bags_train[0][0].shape[0]
            #step_per_epoch = len(bags_train)
            step_per_epoch = len(np.unique(bags))

            lr_step = (step_per_epoch * 40)
            display = (step_per_epoch * display_every)
            max_steps = n_epochs * step_per_epoch
            
            model = ShapeletGenerator(n_prototypes, bag_size, n_classes, features)

            if n_classes == 2:
                output_fn = torch.nn.Sigmoid()
            elif n_classes == 3:
                output_fn = torch.nn.Softmax()
            else:
                output_fn = torch.nn.Linear(n_classes,1)
            
            if n_classes == 2:
                loss = torch.nn.BCEWithLogitsLoss(reduction="mean")
            else:
                loss = torch.nn.CrossEntropyLoss(reduction="mean")
                
            loss = torch.nn.MSELoss(reduction="mean")

            optim1 = torch.optim.Adam([model.prototypes], lr=lr_prot)
            optim2 = torch.optim.Adam(list(model.linear_layer.parameters()),
                        lr=lr_weights)
            total_loss = 0
            correct = 0
            train_loss_hist, eval_loss_hist = [], []
            train_acc_hist, eval_acc_hist = [], []
            eval_aucs = []
            step_hist = []
            time_hist = []

            if use_cuda and torch.cuda.is_available():
                model = model.cuda()

            cont = True
            
            max_stagnation = 0 # number of epochs without improvement to tolerate
            best_prototype = None
            best_score = 1000000000
            i = 0
            
            while i < max_steps and max_stagnation < early_stopping_round:
                i += 1
                np_idx = np.random.choice(bags_train.shape[0], batch_size)
                start_time = time.time()
                batch_inp = bags_train[np_idx]
                targets = torch.Tensor(labels_train[np_idx])
                batch_inp = torch.Tensor(batch_inp[0])
                batch_inp = batch_inp.view(1, batch_inp.shape[0], batch_inp.shape[1])
                if use_cuda and torch.cuda.is_available():
                    targets = targets.cuda()
                    batch_inp = batch_inp.cuda()

                logits, distances = model(batch_inp)
                out = output_fn(logits)

                if n_classes == 2:
                    predicted = (out > 0.5).type(torch.int64)
                elif n_classes == 3:
                    _, predicted = torch.max(out, 1)
                else:
                    predicted = out
                correct += (predicted == targets).type(torch.float32).mean().item()
                
                batch_loss = loss(logits, targets.type(torch.float32))

                prototypes_pairwise = pairwise_dist(model.prototypes, model.prototypes)
                reg_prototypes = prototypes_pairwise.sum()

                weight_reg = 0
                for param in model.linear_layer.parameters():
                    weight_reg += param.norm(p=reg_w).sum()

                reg_loss = reg_lambda_w*weight_reg + reg_lambda_dist*distances.sum() - reg_prototypes*reg_lambda_p
                total_loss += batch_loss
                min_loss = batch_loss + reg_loss
                min_loss.backward()

                optim1.step()
                optim2.step()

                if (i + 1) % lr_step == 0:
                    print("LR DROP!")
                    optims = [optim1, optim2]
                    for o in optims:
                        for p in o.param_groups:
                            p["lr"] = p["lr"] / 2

                if (i + 1) % display == 0:
                    with torch.no_grad():
                        #print("Step : ", str(i + 1), "Loss: ",
                        #total_loss.item() / display, " accuracy: ", correct / (display))
                        train_loss_hist.append(total_loss.item() / display)
                        train_acc_hist.append(correct / display)
                        total_loss = 0
                        correct = 0
                        model = model.eval()
                        e_loss = 0
                        e_acc = 0
                        y_true = []
                        y_score = []
                        loss_list = []

                        for i in range(len(bags_train)):
                            batch_inp = torch.Tensor(bags_train[i])
                            batch_inp = batch_inp.view(1, batch_inp.shape[0],
                                                  batch_inp.shape[1])
                            targets = torch.Tensor([labels_train[i]])
                            logits, distances = model(batch_inp)
                            out = output_fn(logits)

                            if n_classes == 2:
                                predicted = (out > 0.5).type(torch.int64)
                            elif n_classes == 3:
                                _, predicted = torch.max(out, 1)
                            else:
                                predicted = out
                            #y_true.append(targets)
                            #y_score.append(out)
                            correct = (predicted == targets).type(torch.float32).mean().item()
                            e_acc += correct
                            eval_loss = loss(logits, targets.type(torch.float32)).item()
                            e_loss += eval_loss
                            loss_list.append(e_loss)

                        #y_true_list = [x.tolist() for x in y_true]
                        #y_score_list = [x.tolist() for x in y_score]
                        #score_auc = roc_auc_score(y_true_list, y_score_list)
                        #print("Eval Loss: ", e_loss / len(bags_train),
                        #    " Eval Accuracy:", e_acc / len(bags_train), " AUC: ",
                        #score_auc)
                        loss_score = np.mean(loss_list)
                        
                        #if score_auc > best_score:
                        if loss_score < best_score:
                            #best_score = score_auc
                            best_score = loss_score
                            best_prototype = model.prototypes
                            max_stagnation = 0
                        else:
                            max_stagnation += 1
                        
                        #print("max_stagnation ", max_stagnation)
                        eval_loss_hist.append(e_loss / len(bags_train))
                        eval_acc_hist.append(e_acc / len(bags_train))
                        #eval_aucs.append(roc_auc_score(y_true_list, y_score_list))
                        accs.append(e_acc / len(bags_train))
                        step_hist.append(i+1)
                        model = model.train()
    
    return best_prototype, [x for x in model.linear_layer.parameters()]

parameters = [[0.00001, 0.05], [0.00001, 0.05],[0.00001, 0.05], [0.00001, 0.05], [0.00001, 0.05], [1],[1]]

In [31]:
import os
import random
import sys
import time
import math

import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.pipeline import Pipeline
from scipy.stats import mode

class Node:
    def __init__(self):

        self.right = None
        self.left = None
        
        self.prototype = None
        
        self.column = None
        self.threshold = None
        
        self.probas = None
        self.depth = None
        
        self.is_terminal = False
        
class PrototypeTreeClassifier:
    def __init__(self,
                train_features,
                 feature_types = ["min", "max", "mean"], 
                 max_depth = 3, 
                 min_samples_leaf = 1, 
                 min_samples_split = 2, 
                 prototype_count = 1,
                 use_prototype_learner=True,
                 early_stopping_round = 3):

        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        self.prototype_count = prototype_count
        self.feature_types = feature_types
        self.train_features = train_features
        self.use_prototype_learner = use_prototype_learner
        self.Tree = None
        self.early_stopping_round = early_stopping_round
        
    def prototype(self, bags, features, labels, prototype_count):
        if self.use_prototype_learner:
            prototypes, _ = find_prototype(bags, features, labels, self.early_stopping_round)
            check = prototypes.cpu().detach().numpy()

            check.resize(check.shape[1], check.shape[2])
            
            return check
        
        else:
            number_of_rows = features.shape[0]
            random_indices = np.random.choice(number_of_rows, 
                                              size=prototype_count, 
                                              replace=False)
            
            prot = features[random_indices, :]
            if len(prot.shape) == 1:
                prot = prot.reshape(1, prot.shape[0])
            
            return prot

    def nodeProbas(self, y):
        # for each unique label calculate the probability for it
        probas = []

        return np.asarray(np.sum(y)/y.size)

    def features_via_prototype(self, feature_types, features, bag_ids, prototypes):
        distances = self.calculate_distances(features, prototypes)
        
        bin_count  = np.unique(bag_ids, return_counts=True)[1]
        _, index  = np.unique(bag_ids, return_index=True)

        feature_list = []
        for i in range(0, prototypes.shape[0]):
            if "max" in feature_types:
                group_max = np.maximum.reduceat(distances[:, i], index)
                max_vals = np.repeat(group_max, bin_count)
                feature_list.append(max_vals)

            if "min" in feature_types:
                group_min = np.minimum.reduceat(distances[:, i], index)
                min_vals = np.repeat(group_min, bin_count)
                feature_list.append(min_vals)

            if "mean" in feature_types:
                group_mean = np.add.reduceat(distances[:, i], index)
                mean_vals = np.repeat(group_mean/bin_count, bin_count)
                feature_list.append(mean_vals)
        
        return np.array(np.transpose(feature_list))

    def dist1d(self, features, prototypes, distance_type="l2"):
        if distance_type == "l2":

            distance = np.linalg.norm(features - prototypes, axis=1)
        elif distance_type == "l1":
            distance = np.abs(features - prototypes)
            distance = np.sum(distance, axis=1)

        return distance

    def calculate_distances(self, features, prototypes):
        feature_list = []
        
        for i in range(0, prototypes.shape[0]):
            data = self.dist1d(features, prototypes[i], distance_type="l2")
            feature_list.append(data)
        data = np.column_stack(feature_list)

        return data

    def calcBestSplit(self, features, features_via_prototype, labels, bag_ids):
        ids, index  = np.unique(bag_ids, return_index=True)
        
        bdc = tree.DecisionTreeRegressor(
            min_samples_split=2,
            criterion="mae"
        )
        bdc.fit(features_via_prototype[index], labels[index])
        
        threshold = bdc.tree_.threshold[0]
        split_col = bdc.tree_.feature[0]

        features_left = features[features_via_prototype[:,split_col] <= bdc.tree_.threshold[0]]
        features_right = features[features_via_prototype[:,split_col] > bdc.tree_.threshold[0]]

        labels_left = labels[features_via_prototype[:,split_col] <= bdc.tree_.threshold[0]]
        labels_right = labels[features_via_prototype[:,split_col] > bdc.tree_.threshold[0]]

        bag_ids_left = bag_ids[features_via_prototype[:,split_col] <= bdc.tree_.threshold[0]]
        bag_ids_right = bag_ids[features_via_prototype[:,split_col] > bdc.tree_.threshold[0]]

        return split_col, threshold, features_left, features_right, labels_left, labels_right, bag_ids_left, bag_ids_right

    def buildDT(self, features, labels, bag_ids, node):
            '''
            Recursively builds decision tree from the top to bottom
            '''
            # checking for the terminal conditions

            if node.depth >= self.max_depth:
                node.is_terminal = True
                return

            if len(np.unique(bag_ids)) < self.min_samples_split:
                node.is_terminal = True
                return

            if np.unique(labels).shape[0] == 1:
                node.is_terminal = True
                return
            
            node.prototype = self.prototype(bag_ids, features, labels, self.prototype_count)
            features_updated = self.features_via_prototype(self.feature_types, features, bag_ids, node.prototype)
            
            # calculating current split
            (splitCol, 
             thresh, 
             features_left, 
             features_right, 
             labels_left, 
             labels_right, 
             bag_ids_left, 
             bag_ids_right) = self.calcBestSplit(features, 
                                                 features_updated, 
                                                 labels, 
                                                 bag_ids)
            
            if splitCol is None:
                node.is_terminal = True
                return

            if len(np.unique(bag_ids_left)) < self.min_samples_leaf or len(np.unique(bag_ids_right)) < self.min_samples_leaf:
                node.is_terminal = True
                return
            
            node.column = splitCol
            node.threshold = thresh
            
            _, index_left  = np.unique(bag_ids_left, return_index=True)
            _, index_right  = np.unique(bag_ids_right, return_index=True)
            
            # creating left and right child nodes
            node.left = Node()
            node.left.depth = node.depth + 1
            node.left.probas = self.nodeProbas(labels_left[index_left])

            node.right = Node()
            node.right.depth = node.depth + 1
            node.right.probas = self.nodeProbas(labels_right[index_right])

            # splitting recursively
            
            self.buildDT(features_right, labels_right, bag_ids_right, node.right)
            self.buildDT(features_left, labels_left, bag_ids_left, node.left)

    def fit(self, features, labels, bag_ids):
        '''
        Standard fit function to run all the model training
        '''
        self.Tree = Node()
        self.Tree.depth = 1
        
        self.buildDT(features, labels, bag_ids, self.Tree)

    def predictSample(self, features, bag_ids, node):
        '''
        Passes one object through decision tree and return the probability of it to belong to each class
        '''

        # if we have reached the terminal node of the tree
        if node.is_terminal:
            return node.probas

        features_updated = self.features_via_prototype(self.feature_types, features, bag_ids, node.prototype)

        if features_updated[0][node.column] > node.threshold:
            probas = self.predictSample(features, bag_ids, node.right)
        else:
            probas = self.predictSample(features, bag_ids, node.left)

        return probas

    def predict(self, features, bag_ids):
        '''
        Returns the labels for each X
        '''

        if type(features) == pd.DataFrame:
            X = np.asarray(features)

        sort_index = np.argsort(bag_ids)
        bag_ids = bag_ids[sort_index]
        features = features[sort_index]

        features_updated = self.features_via_prototype(self.feature_types, features, bag_ids, self.Tree.prototype)

        index  = np.unique(bag_ids, return_index=True)[1]
        count  = np.unique(bag_ids, return_counts=True)[1]
        index = np.append(index, bag_ids.shape[0])   
        predictions = []

        for i in range(0, len(index) - 1):
            pred = self.predictSample(features[index[i]:index[i+1]], 
                                                bag_ids[index[i]:index[i+1]], 
                                                self.Tree)
            
            pred = np.repeat(pred, count[i])
            predictions = np.concatenate((predictions, pred), axis=0)
        
        return np.asarray(predictions)

class PrototypeForest:
    def __init__(self, size,
                feature_types = ["min", "mean", "max"],
                max_depth = 8, 
                min_samples_leaf = 2, 
                min_samples_split = 2, 
                prototype_count = 1,
                use_prototype_learner = True,
                early_stopping_round = 10):
        self.size = size
        self._trees = []
        self._tuning_trees = []
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.min_samples_split = min_samples_split
        self.prototype_count = prototype_count
        self.use_prototype_learner = use_prototype_learner
        self.early_stopping_round = early_stopping_round
        
    def sample(self, features, labels, bag_ids):
        ids, index  = np.unique(bag_ids, return_index=True)
        group_min = np.minimum.reduceat(labels, index)
        
        if self.size == 1:
            bag_size = math.ceil(group_min.shape[0] * 1)
        else:
            bag_size = math.ceil(group_min.shape[0] * 0.8)
  
        bags_all = np.random.choice(ids, bag_size, replace=False)
        
        df = pd.DataFrame(np.concatenate([train_bag_ids.reshape(train_bag_ids.shape[0],1),
                                          train_labels.reshape(train_labels.shape[0],1)],
                                         axis=1))
        
        indices_all = df[df[0].isin(bags_all)].index.to_numpy()
        inbag_indices = indices_all
        oo_bag_mask = np.ones(labels.shape[0], dtype=bool)
        oo_bag_mask[inbag_indices] = False
        outbag_indices = np.where(oo_bag_mask == 1)
        
        return inbag_indices, outbag_indices

    
    def fit(self, features, labels, bag_ids):
        for i in range(self.size):
            
            (inbag_indices, _) = self.sample(features, labels, bag_ids)
            inbag_features = features[inbag_indices]
            inbag_labels = labels[inbag_indices]
            inbag_bag_ids = bag_ids[inbag_indices]
            tree = PrototypeTreeClassifier(
                max_depth=self.max_depth,
                min_samples_leaf=self.min_samples_leaf,
                min_samples_split=self.min_samples_split,
                prototype_count = self.prototype_count,
                use_prototype_learner = self.use_prototype_learner,
                train_features = inbag_features,
                early_stopping_round = self.early_stopping_round
            )
            tree.fit(inbag_features, inbag_labels, inbag_bag_ids)
            while tree.Tree.right is None:
                tree.fit(inbag_features, inbag_labels, inbag_bag_ids)
            self._trees.append(tree)
            
    def predict(self, features, bag_ids):
        temp = [t.predict(features, bag_ids) for t in self._trees]
        preds = np.transpose(np.array(temp))
        return np.sum(preds, axis=1)/self.size
   
def split_features_labels_bags(data):
    features = data[data.columns[~data.columns.isin([0, 1])]].to_numpy()
    labels = data[0].to_numpy()
    bag_ids = data[1].to_numpy()

    #sort_index = np.argsort(bag_ids)
    #bag_ids = bag_ids[sort_index]
    #features = features[sort_index]
    
    return (features, labels, bag_ids)

def train_test_split(dataset, rep, fold, explained_variance, fit_on_full = False, custom=False, cols=None):
    data = pd.read_csv(f"./datasets_regression/{dataset}.csv", header=None, sep=" ")
    testbags =  pd.read_csv(f"./datasets_regression/{dataset}.csv_rep{rep}_fold{fold}.txt", header=None)
    #data = pd.read_csv(f"./datasets_regression/{dataset}.csv", header=None, sep=" ")
    #testbags =  pd.read_csv(f"./datasets_regression/cv/{dataset}.csv_rep{rep}_fold{fold}.txt", header=None)

    if cols:
        data = data[list(range(cols))]
    
    if custom:
        min_limit = testbags.min()[0]
        max_limit = testbags.max()[0]
        size = testbags.size
        size_pos = size // 2
        pos = list(range(min_limit, min_limit + size_pos))
        neg = list(range(max_limit - size_pos + 1, max_limit + 1))
        testbags = pd.DataFrame([*pos, *neg])
          
    train_data = data[~data[1].isin(testbags[0].tolist())]    
    
    #for i in range(2, 94):
    #    clean_data = train_data[(train_data[i] != 0) & (train_data[i] != -32767)]
    #    mean = clean_data[i].mean()
    #    train_data[(train_data[i] == 0) | (train_data[i] == -32767)] = mean

    test_data = data[data[1].isin(testbags[0].tolist())]
    
    (train_features, train_labels, train_bag_ids) = split_features_labels_bags(train_data)
    (test_features, test_labels, test_bag_ids) = split_features_labels_bags(test_data)
    
    if explained_variance < 1:
        pipe = Pipeline([('pca', PCA(n_components = explained_variance, 
                         svd_solver = "full")), 
         ('scaler', StandardScaler()), ])
    else:
        pipe = Pipeline([('scaler', StandardScaler()), ])
    
    if fit_on_full:
        pipe.fit(data[data.columns[~data.columns.isin([0,1])]].to_numpy())
    else:
        pipe.fit(train_features)

    train_features = pipe.transform(train_features)
    test_features = pipe.transform(test_features)
    
    return (
        train_features, 
        train_labels, 
        train_bag_ids,
        test_features, 
        test_labels,
        test_bag_ids)

def generate_random(lower, upper):
    random_number = random.random()
    random_number = random_number + lower
    random_range = upper - lower
    random_number = random_number*random_range
    return random_number



In [23]:
info_list = []
pred_all = []

for i in range(1, 46):
    col_no = i*2 + 2
    dataset = "WheatYields"
    print(i)
    
    (train_features,
         train_labels,
         train_bag_ids,
         test_features,
         test_labels,
         test_bag_ids) = train_test_split(dataset, 1, 1, 1, fit_on_full = True, cols = col_no)

    model = PrototypeForest(size=2,
                            max_depth=2,
                            min_samples_leaf=2,
                            min_samples_split=4,
                            prototype_count=1,
                            early_stopping_round= 5)

    model.fit(train_features, train_labels, train_bag_ids)

    probas = model.predict(test_features, test_bag_ids)
    
    pred_list = list(zip(probas, test_labels))
    names = dataset.split("_")

    pred_list = [(i, x[0], x[1]) for x in pred_list]

    pred_all.extend(pred_list)

    #pred_df = pd.DataFrame(probas, columns=["prediction"])
    #pred_df.to_csv(f"./performance/prediction_reg_corn_{i}.csv")

    _, index  = np.unique(test_bag_ids, return_index=True)

    score = metrics.mean_absolute_error(test_labels[index], probas[index])

    mean = score/(np.sum(test_labels)/test_labels.size)
    info_list_row = [i, mean]
    
    info_list.append(info_list_row)

perf_df = pd.DataFrame(info_list, columns=["i", "score"])
perf_df.to_csv(f"./performance/performance_learning_{dataset}.csv")

all_df = pd.DataFrame(pred_all, columns=["i", "prediction", "label"])
all_df.to_csv(f"./performance/predictions__{dataset}.csv")

1
> <ipython-input-19-18fe482d550d>(239)find_prototype()
    237 
    238                 import pdb; pdb.set_trace()
--> 239                 logits, distances = model(batch_inp)
    240                 out = output_fn(logits)
    241 

ipdb> exit()


BdbQuit: 

In [14]:
info_list

[[1, 0.3755613497683672]]

In [23]:
info_list = []
pred_all = []

for i in range(1, 46):
    col_no = i*2 + 2
    dataset = "CornYields"
    print(i)
    
    (train_features,
         train_labels,
         train_bag_ids,
         test_features,
         test_labels,
         test_bag_ids) = train_test_split(dataset, 1, 1, 1, fit_on_full = True, cols = col_no)

    model = PrototypeForest(size=100,
                            max_depth=8,
                            min_samples_leaf=2,
                            min_samples_split=4,
                            prototype_count=1,
                            early_stopping_round= 5)

    model.fit(train_features, train_labels, train_bag_ids)

    probas = model.predict(test_features, test_bag_ids)
    
    pred_list = list(zip(probas, test_labels))
    names = dataset.split("_")

    pred_list = [(i, x[0], x[1]) for x in pred_list]

    pred_all.extend(pred_list)

    #pred_df = pd.DataFrame(probas, columns=["prediction"])
    #pred_df.to_csv(f"./performance/prediction_reg_corn_{i}.csv")

    _, index  = np.unique(test_bag_ids, return_index=True)

    score = metrics.mean_absolute_error(test_labels[index], probas[index])

    mean = score/(np.sum(test_labels)/test_labels.size)
    info_list_row = [i, mean]
    
    info_list.append(info_list_row)

perf_df = pd.DataFrame(info_list, columns=["i", "score"])
perf_df.to_csv(f"./performance/performance_{dataset}.csv")

all_df = pd.DataFrame(pred_all, columns=["i", "prediction", "label"])
all_df.to_csv(f"./performance/predictions_{dataset}.csv")

1


FileNotFoundError: [Errno 2] No such file or directory: './datasets_regression/cv/CornYields.csv_rep1_fold1.txt'

In [24]:
def train_test_split(dataset, rep, fold, explained_variance, fit_on_full = False, custom=False, cols=None):
    #data = pd.read_csv(f"./datasets_regression/{dataset}.csv", header=None, sep=" ")
    #testbags =  pd.read_csv(f"./datasets_regression/{dataset}.csv_rep{rep}_fold{fold}.txt", header=None)
    data = pd.read_csv(f"./datasets_regression/syn_new/{dataset}.csv", header=None, sep=",")
    testbags =  pd.read_csv(f"./datasets_regression/syn_new/cv/{dataset}.csv_rep{rep}_fold{fold}.txt", sep=",")
    
    data = np.round(data,2)
    
    if cols:
        data = data[list(range(cols))]
    
    if custom:
        min_limit = testbags.min()[0]
        max_limit = testbags.max()[0]
        size = testbags.size
        size_pos = size // 2
        pos = list(range(min_limit, min_limit + size_pos))
        neg = list(range(max_limit - size_pos + 1, max_limit + 1))
        testbags = pd.DataFrame([*pos, *neg])

    train_data = data[~data[1].isin(testbags["x"].tolist())]    
    
    test_data = data[data[1].isin(testbags["x"].tolist())]
    
    (train_features, train_labels, train_bag_ids) = split_features_labels_bags(train_data)
    (test_features, test_labels, test_bag_ids) = split_features_labels_bags(test_data)
    
    if explained_variance < 1:
        pipe = Pipeline([('pca', PCA(n_components = explained_variance, 
                         svd_solver = "full")), 
         ('scaler', StandardScaler()), ])
    else:
        pipe = Pipeline([('scaler', StandardScaler()), ])
    
    if fit_on_full:
        pipe.fit(data[data.columns[~data.columns.isin([0,1])]].to_numpy())
    else:
        pipe.fit(train_features)

    train_features = pipe.transform(train_features)
    test_features = pipe.transform(test_features)
    
    train_features = np.round(train_features,2)
    test_features = np.round(test_features,2)

    return (
        train_features, 
        train_labels, 
        train_bag_ids,
        test_features, 
        test_labels,
        test_bag_ids)


In [9]:
def train_test_split(dataset, rep, fold, explained_variance, fit_on_full = False, custom=False, cols=None):
    #data = pd.read_csv(f"./datasets_regression/{dataset}.csv", header=None, sep=" ")
    #testbags =  pd.read_csv(f"./datasets_regression/{dataset}.csv_rep{rep}_fold{fold}.txt", header=None)
    data = pd.read_csv(f"./datasets_regression/syn_new/{dataset}.csv", header=None, sep=",")
    testbags =  pd.read_csv(f"./datasets_regression/syn_new/cv/{dataset}.csv_rep{rep}_fold{fold}.txt", sep=",")
    
    data = np.round(data,2)
    
    if cols:
        data = data[list(range(cols))]
    
    if custom:
        min_limit = testbags.min()[0]
        max_limit = testbags.max()[0]
        size = testbags.size
        size_pos = size // 2
        pos = list(range(min_limit, min_limit + size_pos))
        neg = list(range(max_limit - size_pos + 1, max_limit + 1))
        testbags = pd.DataFrame([*pos, *neg])

    train_data = data[~data[1].isin(testbags["x"].tolist())]    
    
    test_data = data[data[1].isin(testbags["x"].tolist())]
    
    (train_features, train_labels, train_bag_ids) = split_features_labels_bags(train_data)
    (test_features, test_labels, test_bag_ids) = split_features_labels_bags(test_data)
    
    if explained_variance < 1:
        pipe = Pipeline([('pca', PCA(n_components = explained_variance, 
                         svd_solver = "full")), 
         ('scaler', StandardScaler()), ])
    else:
        pipe = Pipeline([('scaler', StandardScaler()), ])
    
    if fit_on_full:
        pipe.fit(data[data.columns[~data.columns.isin([0,1])]].to_numpy())
    else:
        pipe.fit(train_features)

    train_features = pipe.transform(train_features)
    test_features = pipe.transform(test_features)
    
    train_features = np.round(train_features,2)
    test_features = np.round(test_features,2)

    return (
        train_features, 
        train_labels, 
        train_bag_ids,
        test_features, 
        test_labels,
        test_bag_ids)

import os

folders = os.listdir("/home/erdemb/libs/mil/datasets2/")
datasets = [x for x in folders if x != "cv"]
datasets = [x.split(".")[0] for x in datasets]

info_list = []
pred_all = []

dataset = ["nBag_10_nFeat_2_nInsPerBag_2"]

for dataset in datasets:
    for rep in range(1, 6):
        for fold in range(1, 11):
            print(f"dataset {dataset}, rep {rep}, fold {fold}")
            (train_features,
                 train_labels,
                 train_bag_ids,
                 test_features,
                 test_labels,
                 test_bag_ids) = train_test_split(dataset, rep, fold, 1, fit_on_full = True)

            model = PrototypeForest(size=1,
                                    max_depth=2,
                                    min_samples_leaf=2,
                                    min_samples_split=4,
                                    prototype_count=1,
                                    early_stopping_round= 5)

            model.fit(train_features, train_labels, train_bag_ids)

            probas = model.predict(test_features, test_bag_ids)
            
            pred_list = list(zip(probas, test_labels))
            names = dataset.split("_")
            
            pred_list = [(names[1], names[3], names[5], rep, fold, x[0], x[1]) for x in pred_list]
            
            pred_all.extend(pred_list)

            #pred_df = pd.DataFrame(probas, columns=["prediction"])
            #pred_df.to_csv(f"./performance/prediction_{dataset}_rep_{rep}_fold_{fold}.csv")

            _, index  = np.unique(test_bag_ids, return_index=True)

            score = metrics.mean_absolute_error(test_labels[index], probas[index])

            mean = score/(np.sum(test_labels)/test_labels.size)
            info_list_row = [dataset, rep, fold, mean]

            info_list.append(info_list_row)

all_df = pd.DataFrame(pred_all, columns=["nBag", "nFeat","nInsPerBag","rep", "fold", "prediction", "label"])
           
perf_df = pd.DataFrame(info_list, columns=["dataset", "rep", "fold", "score"])

#perf_df.to_csv(f"./performance/performance_learning_synthetic.csv")

#all_df.to_csv(f"./performance/predictions_learning_synthetic.csv")

dataset nBag_10_nFeat_2_nInsPerBag_2, rep 1, fold 1


FileNotFoundError: [Errno 2] No such file or directory: './datasets_regression/syn_new/nBag_10_nFeat_2_nInsPerBag_2.csv'

In [42]:
import os

folders = os.listdir("/home/erdemb/libs/mil/datasets_regression/syn_new/")
datasets = [x for x in folders if x != "cv"]
datasets = [x.split(".")[0] for x in datasets]

info_list = []
pred_all = []

for dataset in datasets:
    for rep in range(1, 6):
        for fold in range(1, 11):
            print(f"dataset {dataset}, rep {rep}, fold {fold}")
            (train_features,
                 train_labels,
                 train_bag_ids,
                 test_features,
                 test_labels,
                 test_bag_ids) = train_test_split(dataset, rep, fold, 1, fit_on_full = True)

            model = PrototypeForest(size=100,
                                    max_depth=4,
                                    min_samples_leaf=2,
                                    min_samples_split=4,
                                    prototype_count=1,
                                    early_stopping_round= 5)

            model.fit(train_features, train_labels, train_bag_ids)

            probas = model.predict(test_features, test_bag_ids)
            
            pred_list = list(zip(probas, test_labels))
            names = dataset.split("_")
            
            pred_list = [(names[1], names[3], names[5], rep, fold, x[0], x[1]) for x in pred_list]
            
            pred_all.extend(pred_list)

            #pred_df = pd.DataFrame(probas, columns=["prediction"])
            #pred_df.to_csv(f"./performance/prediction_{dataset}_rep_{rep}_fold_{fold}.csv")

            _, index  = np.unique(test_bag_ids, return_index=True)

            score = metrics.mean_absolute_error(test_labels[index], probas[index])

            mean = score/(np.sum(test_labels)/test_labels.size)
            info_list_row = [dataset, rep, fold, mean]

            info_list.append(info_list_row)

all_df = pd.DataFrame(pred_all, columns=["nBag", "nFeat","nInsPerBag","rep", "fold", "prediction", "label"])
           
perf_df = pd.DataFrame(info_list, columns=["dataset", "rep", "fold", "score"])

perf_df.to_csv(f"./performance/performance_multitree_synthetic.csv")

all_df.to_csv(f"./performance/predictions_multitree_synthetic.csv")

dataset nBag_25_nFeat_2_nInsPerBag_2, rep 1, fold 1


FileNotFoundError: [Errno 2] No such file or directory: './datasets2/nBag_25_nFeat_2_nInsPerBag_2.csv'

In [34]:
def train_test_split(dataset, rep, fold, explained_variance, fit_on_full = False, custom=False, cols=None):
    data = pd.read_csv(f"./datasets2/{dataset}.csv", header=None)
    testbags =  pd.read_csv(f"./datasets2/{dataset}.csv_rep{rep}_fold{fold}.txt", header=None)
    if custom:
        min_limit = testbags.min()[0]
        max_limit = testbags.max()[0]
        size = testbags.size
        size_pos = size // 2
        pos = list(range(min_limit, min_limit + size_pos))
        neg = list(range(max_limit - size_pos + 1, max_limit + 1))
        testbags = pd.DataFrame([*pos, *neg])
    
    train_data = data[~data[1].isin(testbags[0].tolist())]
    test_data = data[data[1].isin(testbags[0].tolist())]
    (train_features, train_labels, train_bag_ids) = split_features_labels_bags(train_data)
    (test_features, test_labels, test_bag_ids) = split_features_labels_bags(test_data)
    
    if explained_variance < 1:
        pipe = Pipeline([('pca', PCA(n_components = explained_variance,
                         svd_solver = "full")),
         ('scaler', StandardScaler()), ])
    else:
        pipe = Pipeline([('scaler', StandardScaler()), ])
    if fit_on_full:
        pipe.fit(data[data.columns[~data.columns.isin([0,1])]].to_numpy())
    else:
        pipe.fit(train_features)
    #train_features = pipe.transform(train_features)
    #test_features = pipe.transform(test_features)
    return (
        train_features,
        train_labels,
        train_bag_ids,
        test_features,
        test_labels,
        test_bag_ids)

import os

folders = os.listdir("/home/erdemb/libs/mil/datasets2/")
datasets = [x for x in folders if x != "cv"]
datasets = [x.split(".")[0] for x in datasets]

info_list = []
pred_all = []

datasets = ["nBag_10_nFeat_2_nInsPerBag_2"]

for dataset in datasets:
    for rep in range(1, 6):
        for fold in range(1, 11):
            print(f"dataset {dataset}, rep {rep}, fold {fold}")
            (train_features,
                 train_labels,
                 train_bag_ids,
                 test_features,
                 test_labels,
                 test_bag_ids) = train_test_split(dataset, rep, fold, 1, fit_on_full = False)

            model = PrototypeForest(size=1,
                                    max_depth=2,
                                    min_samples_leaf=2,
                                    min_samples_split=4,
                                    prototype_count=1,
                                    early_stopping_round= 5)

            model.fit(train_features, train_labels, train_bag_ids)
            
            probas = model.predict(test_features, test_bag_ids)
            
            pred_list = list(zip(probas, test_labels))
            names = dataset.split("_")
            
            pred_list = [(names[1], names[3], names[5], rep, fold, x[0], x[1]) for x in pred_list]
            
            pred_all.extend(pred_list)

            #pred_df = pd.DataFrame(probas, columns=["prediction"])
            #pred_df.to_csv(f"./performance/prediction_{dataset}_rep_{rep}_fold_{fold}.csv")

            _, index  = np.unique(test_bag_ids, return_index=True)

            score = metrics.mean_absolute_error(test_labels[index], probas[index])

            mean = score/(np.sum(test_labels)/test_labels.size)
            info_list_row = [dataset, rep, fold, mean]

            info_list.append(info_list_row)

all_df = pd.DataFrame(pred_all, columns=["nBag", "nFeat","nInsPerBag","rep", "fold", "prediction", "label"])
           
perf_df = pd.DataFrame(info_list, columns=["dataset", "rep", "fold", "score"])

#perf_df.to_csv(f"./performance/performance_learning_synthetic.csv")

#all_df.to_csv(f"./performance/predictions_learning_synthetic.csv")

dataset nBag_10_nFeat_2_nInsPerBag_2, rep 1, fold 1
> <ipython-input-30-f265749327ad>(74)get_output()
     72         dist = self.pairwise_distances(batch_inp, self.prototypes)
     73         import pdb; pdb.set_trace()
---> 74         min_dist = dist.min(dim=1)[0]
     75         max_dist = dist.max(dim=1)[0]
     76         mean_dist = dist.mean(dim=1)

ipdb> exit()


BdbQuit: 

In [19]:
perf_df

,dataset,rep,fold,score
0,nBag_10_nFeat_2_nInsPerBag_2,1,1,0.476602
1,nBag_10_nFeat_2_nInsPerBag_2,1,2,0.328550
2,nBag_10_nFeat_2_nInsPerBag_2,1,3,0.138767
3,nBag_10_nFeat_2_nInsPerBag_2,1,4,1.141307
4,nBag_10_nFeat_2_nInsPerBag_2,1,5,0.023132
5,nBag_10_nFeat_2_nInsPerBag_2,1,6,0.353076
6,nBag_10_nFeat_2_nInsPerBag_2,1,7,0.518415
7,nBag_10_nFeat_2_nInsPerBag_2,1,8,0.400337
8,nBag_10_nFeat_2_nInsPerBag_2,1,9,0.691329
9,nBag_10_nFeat_2_nInsPerBag_2,1,10,0.954532
